# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [16]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [17]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [7]:
# !pip install langchain_community

In [10]:
# !pip install langchain langchain-openai

In [18]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [31]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=1)


In [32]:
prompt = ChatPromptTemplate.from_template( "Describe the product: {product}"
 
)

In [33]:

chain = LLMChain(llm=llm, prompt=prompt)

In [34]:
product = "Playstation 5"
chain.run(product)

'The PlayStation 5 is a video game console developed by Sony Interactive Entertainment. It is the successor to the PlayStation 4 and is the fifth installment in the PlayStation console series. The console features cutting-edge hardware including a custom-built processor, high-speed SSD storage, and support for 4K gaming and ray tracing technology. The PlayStation 5 also boasts a new controller design with haptic feedback and adaptive triggers for more immersive gameplay experiences. It supports a wide range of games from various genres and offers online multiplayer capabilities through the PlayStation Network. Overall, the PlayStation 5 delivers a next-generation gaming experience with stunning graphics, fast loading times, and innovative features.'

## SimpleSequentialChain

In [35]:
from langchain.chains import SimpleSequentialChain

In [37]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Give a short marketing-style description of the product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [38]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following description, suggest a creative and catchy product name: {text}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [39]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [40]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Experience the next generation of gaming with the Playstation 5. Boasting lightning-fast load times, stunning graphics, and immersive gameplay, the PS5 takes your gaming experience to new heights. With cutting-edge technology and innovative features, the PS5 is the ultimate console for any gamer looking to elevate their gaming experience.
GameXcelerator

> Finished chain.


'GameXcelerator'

In [41]:
product = "Iphone"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
The iPhone is a revolutionary smartphone that seamlessly combines sleek design, cutting-edge technology, and unparalleled user experience. With its stunning Retina display, powerful camera, and secure operating system, the iPhone offers endless possibilities for work, play, and everything in between. Stay connected, organized, and entertained like never before with the iPhone. Experience the ultimate in mobile innovation with the iPhone.
InnoTech Visionary Phone

> Finished chain.


'InnoTech Visionary Phone'

In [42]:
product = "Pencil"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Introducing the classic writing tool that never goes out of style - the Pencil! Perfect for jotting down notes, sketching out ideas, or simply adding a touch of creativity to your everyday tasks. With its sleek design and smooth writing capabilities, the Pencil is a must-have accessory for students, artists, and professionals alike. Upgrade your writing experience with the timeless elegance of the Pencil.
ElegaGraph Pencil

> Finished chain.


'ElegaGraph Pencil'

**Repeat the above twice for different products**

## SequentialChain

In [59]:
from langchain.chains import SequentialChain

In [60]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate the following product review to English:\n\n{review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="english_review"
                    )


In [61]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize this English review:\n\n{english_review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [62]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following English review into Spanish:\n\n{english_review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="translated_review"
                      )


In [65]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Write a follow-up message based on this English summary:\n\n{english_review}\n\n"
        "and its translation in Spanish:\n\n{translated_review}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="follow_up"
                     )


In [66]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=["english_review", "summary", "translated_review", "follow_up"],
    verbose=True
)

In [67]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'english_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer is unimpressed with the taste of the product, finding it mediocre. They also note that the foam doesn't hold well and question if the product may be old or counterfeit compared to ones bought in stores which have a better taste.",
 'translated_review': 'Encuentro que el sabor es mediocre. La espuma no se mantiene, es extraño. Compro los mismos en tiendas y el sabor es mucho mejor... ¿Lote viejo o falsificación?',
 'follow_up': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?\n\nEncuentro que el sabor es medio

In [68]:
review = df.Review[3]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'english_review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great.',
 'summary': 'The reviewer highly recommends these throw pillow fillers from Amazon, as they fluff up well in the dryer and are superior to other cheap and flat options they have tried.',
 'translated_review': 'Estos son los mejores rellenos de almohadas en Amazon. He probado varios otros y todos son baratos y planos sin importar cuánto los esponjes. Una vez que los metes en la secadora después de quitarlos del material de envío sellado al 

In [69]:
review = df.Review[1]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'english_review': 'Me encantó la bolsa impermeable, aunque la abertura estaba hecha de plástico duro. No sé si se rompería fácilmente. Pero no pude encender mi teléfono una vez que estaba en la bolsa.',
 'summary': 'The reviewer loved the waterproof bag but was unsure about the durability of the hard plastic opening. They were unable to turn on their phone once it was inside the bag.',
 'translated_review': "I loved the waterproof bag, although the opening was made of hard plastic. I'm not sure if it would break easily. But I couldn't turn on my phone once it was in the bag. \n\nMe encantó la bolsa impermeable, aunque la abertura estaba hecha de plástico duro. No estoy segura si se rompería fácilmente. Pero no pude encender mi teléfono una vez que estaba en la bolsa.",
 'follow_up': '¡Hola! Gracia

**Repeat the above twice for different products or reviews**

## Router Chain

In [70]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [71]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [72]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [73]:
llm = ChatOpenAI(temperature=0)

In [74]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [75]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [76]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [77]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [78]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [79]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power."

In [80]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [81]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


"Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells, maintaining the integrity and continuity of the organism's genetic code.\n\nAdditionally, DNA serves as a storage system for genetic information that can be accessed and utilized by cells as needed. This allows for the regulation of gene expression, the repair of damaged DNA, and the adaptation to changing environmental conditions.\n\nIn summary, every cell in our body contains DNA because it is essential for the pro

In [82]:
chain.run("Which years were the aztec empire?")



> Entering new MultiPromptChain chain...
History: {'input': 'Which years were the Aztec Empire?'}
> Finished chain.


'The Aztec Empire existed from 1428 to 1521.'

In [83]:
chain.run("What is the best history fact that you know?")



> Entering new MultiPromptChain chain...
History: {'input': 'What is the best history fact that you know?'}
> Finished chain.


"One of the most fascinating historical facts that I know is the story of the Great Wall of China. Built over centuries by various Chinese dynasties, the Great Wall stretches over 13,000 miles and is one of the most impressive architectural feats in history. It was constructed to protect China from invasions by nomadic tribes and other enemies, and its sheer size and complexity are a testament to the ingenuity and determination of the ancient Chinese people. The Great Wall remains a symbol of China's rich history and cultural heritage, and is a must-see destination for history enthusiasts around the world."

**Repeat the above at least once for different inputs and chains executions - Be creative!**